Se obtienen los datos de la db del sniim

In [3]:
import psycopg2k
import numpy as np
import pandas as pd
from datetime import datetime

#Datos para obtener un mercado especifico, estos son lo que van a variar al momento de actualizar los datos en la tabla
sourceid = 1 
endid = 40
potatoid = 1

records = []

try:
    connection = psycopg2.connect(user = "postgres", password = "r351d3nc14501", host = "127.0.0.1", port = "5432", database = "potatoe_markets")
    cursor = connection.cursor()
    
    postgreSQL_select_Query = "SELECT P.sniimdatedate, P.avgprice FROM Price P INNER JOIN Market M ON P.SourceMarketID = M.MarketID INNER JOIN Market MM ON P.endmarketid = mm.marketid INNER JOIN Potato PP on p.potatoid = pp.potatoid WHERE m.marketid = {} AND MM.marketid = {} AND p.SNIIMPRESENTATION = 'CALCULADO' AND pp.potatoid = {} AND P.sniimdatedate > (select sniimdate from normalized_price where sourcemarketid = {} and endmarketid = {} order by sniimdate desc limit 1) ORDER BY SNIIMDATEDATE".format(sourceid, endid, potatoid, sourceid, endid)
        
    cursor.execute(postgreSQL_select_Query)
    print("Selecting rows from mobile table using cursor.fetchall")
    value_records = cursor.fetchall()

    print("Save each column value into the correspondent  array")
    for row in value_records:
        records.append(tuple((row[0], row[1])))
    
except (Exception, psycopg2.Error) as error :
    print ("Error while fetching data from PostgreSQL", error)
finally:
    #closing database connection.
    if(connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


Selecting rows from mobile table using cursor.fetchall
Save each column value into the correspondent  array
PostgreSQL connection is closed


Se procesan los datos para rellenar fechas faltantes se guardan en la tabla datos procesados

In [4]:
dates = []
prices = []
for con in range(0, len(records)):
    dates.append(pd.Timestamp(records[con][0]))
    prices.append(records[con][1]/100)
dates = pd.Index(dates)
series = pd.Series(prices, dates)
df = pd.DataFrame({1:series})
#print(df)
df = df.asfreq('D')
df = df.reset_index()
df.columns = range(df.shape[1])

for index,row in df.iterrows():
    row[1] = pd.to_numeric(row[1], errors='coerce')
    if row[0].date().weekday() == 5 or row[0].date().weekday() == 6 :
        df = df.drop([index],axis=0)
        continue

df = df.interpolate()
print(df)

records_to_insert = []

for index,row in df.iterrows():
    records_to_insert.append(tuple((row[0], row[1], sourceid, endid, potatoid)))

#print(records_to_insert)

def bulkInsert(records):
    try:
        connection = psycopg2.connect(user = "postgres", password = "r351d3nc14501", host = "127.0.0.1", port = "5432", database = "potatoe_markets")
        cursor = connection.cursor()
        sql_insert_query = """ INSERT INTO normalized_price (sniimdate, price, sourcemarketid, endmarketid, potatoid) VALUES (%s, %s, %s, %s, %s) """
        # executemany() to insert multiple rows rows
        result = cursor.executemany(sql_insert_query, records)
        connection.commit()
        print(cursor.rowcount, "Record inserted successfully into mobile table")
    except (Exception, psycopg2.Error) as error:
        print("Failed inserting record into mobile table {}".format(error))
    finally:
        # closing database connection.
        if (connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

bulkInsert(records_to_insert)


           0     1
0 2019-09-10  14.0
